In [24]:
from math import log
import operator

def calcShannonEnt(dataSet):  # 计算数据的熵(entropy)
    numEntries=len(dataSet)  # 数据条数
    labelCounts={}
    for featVec in dataSet:
        currentLabel=featVec[-1] # 每行数据的最后一个字（类别）
        # ! 下列三行为过于繁琐的数据统计方法。dislike
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1  # 统计有多少个类以及每个类的数量
        # * 更加被喜欢的方法如下：
        # * labelCounts[currentLabel] = labelCounts.setdefault(currentLabel, 0) + 1
    shannonEnt=0    # shannonEnt 意思是 shannon entropy 香农熵
    for key in labelCounts:
        # * 熵是信息论中最重要和基础的概念，是概率分布的泛函，
        # * 表示随机变量不确定性的大小。不确定性越大，熵越大。
        # * 根据使用环境的不同、泛函的不同，熵分为：自信息、信息熵、联合熵、条件熵、相对熵、互信息
        prob=float(labelCounts[key])/numEntries # 计算单个类的熵值
        # * 信息量公式 $$ I(x)=-\log_{2}{p(x)} $$
        shannonEnt-=prob*log(prob,2) # 累加每个类的熵值
    return shannonEnt

# * createDataSet 方法创造了两个数据矩阵：
# * 其中一个 dataSet 是数据集，是一个二维的数据。存储的内容是数据集内容
# * 另一个 labels 是特征集，是一个一维向量数据。存储的内容是特征值
# * createDataSet 通过 return 对上述两个数据矩阵进行了返回
def createDataSet():    # 创造示例数据
    dataSet = [['长', '粗', '男'],
               ['短', '粗', '男'],
               ['短', '粗', '男'],
               ['长', '细', '女'],
               ['短', '细', '女'],
               ['短', '粗', '女'],
               ['长', '粗', '女'],
               ['长', '粗', '女']]
    labels = ['头发','声音']  #两个特征
    return dataSet,labels

# * splitDataSet：
# @param :  dataSet  list[list]     数据集内容
#           axis     int            某一列的下标
#           value    any            想要从第 axis 列提取的内容的特征值
# * 根据 axis 和 value 提取 dataSet 的第 axis 列内容为 value 的所有行的除了 axis 之外的数据（切分）
# * 函数返回了一个数据矩阵，内容是 axis 为 value 的所有行的其他值的部分
def splitDataSet(dataSet,axis,value): # 按某个特征分类后的数据
    retDataSet=[]
    # ? 也许可以使用 
    # ? retDataSet = [(featVec[:axis] + featVec[axis+1:]) for featVec in dataSet if featVec[axis]==value]
    for featVec in dataSet:
        if featVec[axis]==value:
            # ! 略加繁琐的列表切分方式，可以使用 featVec[:axis] + featVec[axis+1:]。dislike
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

# * 循环整个数据集的特征，有几个特征循环几次，循环过程中根据特征进行分类，对分类的数据求熵
# * 熵和原始数据的熵之间的差值越大，证明对应的这个分类的特征越接近于最佳特征（或者就是最佳特征）
# * 循环结束后返回最佳特征
def chooseBestFeatureToSplit(dataSet):  # 选择最优的分类特征
    numFeatures = len(dataSet[0])-1
    baseEntropy = calcShannonEnt(dataSet)  # 原始的熵
    bestInfoGain = 0
    bestFeature = -1
    for i in range(numFeatures):
        # * 创建特征列表
        featList = [example[i] for example in dataSet]
        # * 创建特征的集合
        uniqueVals = set(featList)
        newEntropy = 0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prob =len(subDataSet)/float(len(dataSet))
            newEntropy +=prob*calcShannonEnt(subDataSet)  # 按特征分类后的熵
        infoGain = baseEntropy - newEntropy  # 原始熵与按特征分类后的熵的差值
        if (infoGain>bestInfoGain):   # 若按某特征划分后，熵值减少的最大，则次特征为最优分类特征
            bestInfoGain=infoGain
            bestFeature = i
    return bestFeature

def majorityCnt(classList):    #按分类后类别数量排序，比如：最后分类为2男1女，则判定为男；
    classCount={}
    for vote in classList:
        # ! 下面三行依旧是不喜欢的词频统计写法
        # * 以下判断类似于一个投票，根据决策树对其进行投票，最后根据投票结果判定预测结果
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet, labels):
    # 把每个数据的类别存入一个单独的类别列表 classList
    classList=[example[-1] for example in dataSet]  # 类别：男或女
    # 特殊终止条件和递归终止条件
    if classList.count(classList[0])==len(classList):
        return classList[0]
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    # 查找最优分类特征
    bestFeat=chooseBestFeatureToSplit(dataSet)
    bestFeatLabel=labels[bestFeat]
    # 使用递归的方法创建决策树
    myTree={bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues=[example[bestFeat] for example in dataSet]
    uniqueVals=set(featValues)
    print(uniqueVals)
    for value in uniqueVals:
        subLabels=labels[:]
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
        return myTree

if __name__=='__main__':
    dataSet, labels = createDataSet()
    print(createTree(dataSet, labels))

{'细', '粗'}
{'声音': {'细': '女'}}


In [13]:
dataSet, labels = createDataSet()
bestFeature = chooseBestFeatureToSplit(dataSet)
t = splitDataSet(dataSet, bestFeature)

1

In [15]:
ls = ['男', '男', '女', '男', '女']
majorityCnt(ls)

'男'